In [ ]:
import os
import tempfile
from pathlib import Path

from rich import print as rprint
from utils import create_mock_circuit_dir

from entitysdk import Client, ProjectContext, models, types

entitycore_api_url = "http://127.0.0.1:8000"
project_context = ProjectContext(
    virtual_lab_id="a98b7abc-fc46-4700-9e3d-37137812c730",
    project_id="0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6",
)
token = os.getenv("ACCESS_TOKEN", "XXX")
client = Client(api_url=entitycore_api_url, project_context=project_context, token_manager=token)

## Create a subject

In [ ]:
species = client.search_entity(
    entity_type=models.Species, query={"name__ilike": "Mus musculus"}
).first()

subject = models.Subject(
    name="my-subject", description="my-subject-description", sex="male", species=species
)

subject = client.register_entity(subject)

## Create circuit model

In [ ]:
brain_region = client.search_entity(entity_type=models.BrainRegion, query={"acronym": "CB"}).first()


circuit = models.Circuit(
    name="my-circuit",
    description="my-circuit",
    subject=subject,
    brain_region=brain_region,
    number_synapses=2,
    number_neurons=5,
    number_connections=10,
    scale="microcircuit",
    build_category="em_reconstruction",
)

## Register circuit

In [ ]:
registered_circuit = client.register_entity(circuit)

In [ ]:
rprint(registered_circuit)

## Upload directory

In [ ]:
with tempfile.TemporaryDirectory() as tdir:
    # create a hierarchy of files to upload
    create_mock_circuit_dir(tdir)

    files = {str(path.relative_to(tdir)): path for path in Path(tdir).rglob("*") if path.is_file()}

    directory_asset = client.upload_directory(
        entity_id=registered_circuit.id,
        entity_type=models.Circuit,
        name="circuit",
        label=types.AssetLabel.sonata_circuit,
        paths=files,
    )

## List directory

In [ ]:
files = client.list_directory(
    entity_id=registered_circuit.id, entity_type=models.Circuit, asset_id=directory_asset.id
)

In [ ]:
rprint(files)

## Download directory

In [ ]:
with tempfile.TemporaryDirectory() as tdir:
    client.download_directory(
        entity_id=registered_circuit.id,
        entity_type=models.Circuit,
        asset_id=directory_asset.id,
        output_path=Path(tdir),
    )
    files = {str(path.relative_to(tdir)): path for path in Path(tdir).rglob("*") if path.is_file()}
    rprint(files)

## Fetch Circuit

In [ ]:
fetched = client.get_entity(entity_id=registered_circuit.id, entity_type=models.Circuit)

In [ ]:
rprint(fetched)